In [61]:
import pandas as pd
import numpy as np
from pathlib import Path
from copy import copy
from tqdm import tqdm
import re

In [2]:
PATH_RAW = Path('../data/raw')

# Loading Data

In [48]:
def proceed_passport(file_path: str) -> [dict]:
    # Using readlines() 
    file_day_4 = open(file_path, 'r') 
    day_4_lines = file_day_4.readlines() 
    
    result = []
    current_passport_dict = {}

    for line in day_4_lines:
        if line=='\n':
            result.append(current_passport_dict)
            current_passport_dict = {}
        else:
            line = line.replace('\n', '')
            parameters = line.split(' ')
            for parameter in parameters:
                parameter = parameter.split(':')
                current_passport_dict[parameter[0]] = parameter[1]
    
    result.append(current_passport_dict)
                
    return result

In [49]:
day_4_data = proceed_passport(file_path=PATH_RAW / 'day_4.txt')

# Step 1

--- Day 4: Passport Processing ---

You arrive at the airport only to realize that you grabbed your North Pole Credentials instead of your passport. While these documents are extremely similar, North Pole Credentials aren't issued by a country and therefore aren't actually valid documentation for travel in most of the world.

It seems like you're not the only one having problems, though; a very long line has formed for the automatic passport scanners, and the delay could upset your travel itinerary.

Due to some questionable network security, you realize you might be able to solve both of these problems at the same time.

The automatic passport scanners are slow because they're having trouble detecting which passports have all required fields. The expected fields are as follows:

byr (Birth Year)
iyr (Issue Year)
eyr (Expiration Year)
hgt (Height)
hcl (Hair Color)
ecl (Eye Color)
pid (Passport ID)
cid (Country ID)
Passport data is validated in batch files (your puzzle input). Each passport is represented as a sequence of key:value pairs separated by spaces or newlines. Passports are separated by blank lines.

Here is an example batch file containing four passports:

ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in
The first passport is valid - all eight fields are present. The second passport is invalid - it is missing hgt (the Height field).

The third passport is interesting; the only missing field is cid, so it looks like data from North Pole Credentials, not a passport at all! Surely, nobody would mind if you made the system temporarily ignore missing cid fields. Treat this "passport" as valid.

The fourth passport is missing two fields, cid and byr. Missing cid is fine, but missing any other field is not, so this passport is invalid.

According to the above rules, your improved system would report 2 valid passports.

Count the number of valid passports - those that have all required fields. Treat cid as optional. In your batch file, how many passports are valid?

In [57]:
def count_invalid_passports(passport_dict_list: [dict], cid=False) -> int:

    result = []
    passport_required_fields = ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']

    if cid:
        passport_required_fields += ['cid']
    
    for passport in tqdm(passport_dict_list):

        valid = True
        for required_field in passport_required_fields:
            if required_field not in passport.keys():
                valid = False


        result.append(valid)

    nb_of_valid = np.sum(result)
    return nb_of_valid

In [58]:
count_invalid_passports(passport_dict_list=day_4_data)

100%|██████████| 260/260 [00:00<00:00, 513668.88it/s]


204

# Step 2

--- Part Two ---

The line is moving more quickly now, but you overhear airport security talking about how passports with invalid data are getting through. Better add some data validation, quick!

You can continue to ignore the cid field, but each other field has strict rules about what values are valid for automatic validation:

byr (Birth Year) - four digits; at least 1920 and at most 2002.
iyr (Issue Year) - four digits; at least 2010 and at most 2020.
eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
hgt (Height) - a number followed by either cm or in:
If cm, the number must be at least 150 and at most 193.
If in, the number must be at least 59 and at most 76.
hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
pid (Passport ID) - a nine-digit number, including leading zeroes.
cid (Country ID) - ignored, missing or not.
Your job is to count the passports where all required fields are both present and valid according to the above rules. Here are some example values:

byr valid:   2002
byr invalid: 2003

hgt valid:   60in
hgt valid:   190cm
hgt invalid: 190in
hgt invalid: 190

hcl valid:   #123abc
hcl invalid: #123abz
hcl invalid: 123abc

ecl valid:   brn
ecl invalid: wat

pid valid:   000000001
pid invalid: 0123456789
Here are some invalid passports:

eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007
Here are some valid passports:

pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719
Count the number of valid passports - those that have all required fields and valid values. Continue to treat cid as optional. In your batch file, how many passports are valid?

In [178]:
def byr_rule(value: str, status: bool) -> bool:
    
    byr = value
    try:
        if len(byr)==4:
            byr = int(value)
            if ((byr<1920) | (byr>2002)):
                status = False 
        else:
            status = False
    except:
        status = False
    return status

def iyr_rule(value: str, status: bool) -> bool:
    
    iyr = value
    try:
        if len(iyr)==4:
            iyr = int(value)
            if ((iyr<2010) | (iyr>2020)):
                status = False 
        else:
            status = False
    except:
        status = False
        
    return status

def eyr_rule(value: str, status: bool) -> bool:
    
    eyr = value
    try:
        if len(eyr)==4:
            eyr = int(value)
            if ((eyr<2020) | (eyr>2030)):
                status = False
        else:
            status = False
    except:
        status = False
        
    return status

def hgt_rule(value: str, status: bool) -> bool:
    
    pattern = r"((^\d{2}in)|(^\d{3}cm))"
    
    if not re.match(pattern, value):
        status = False
    else:
        digit_pattern = r"^\d{2,3}"
        digit = int(re.match(pattern=digit_pattern, string=value).group())
        
        if (('cm' in value) & ((digit<150) | (digit>193))):
            status = False
        elif (('in' in value) & ((digit<59) | (digit>76))):
            status = False
            
    return status

def hcl_rule(value: str, status: bool) -> bool:
    
    pattern = r"(^#([0-9]|[a-f]){6}$)"
    
    if not re.match(pattern, value):
        status = False
    return status

def ecl_rule(value: str, status: bool) -> bool:
    
    if len(value)==3:
        if value not in ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']:
            status = False
    else:
        status = False
    
    return status

def pid_rule(value: str, status: bool) -> bool:
    
    pattern = r"(^[0-9]{9}$)"
    
    if not re.match(pattern, value):
        status = False
    return status

Tests:

In [200]:
assert byr_rule(value='20202', status=True)==False
assert byr_rule(value='1919', status=True)==False
assert byr_rule(value='1920', status=True)==True
assert byr_rule(value='2002', status=True)==True
assert byr_rule(value='2003', status=True)==False
assert byr_rule(value='1950', status=True)==True

assert iyr_rule(value='20202', status=True)==False
assert iyr_rule(value='2009', status=True)==False
assert iyr_rule(value='2010', status=True)==True
assert iyr_rule(value='2020', status=True)==True
assert iyr_rule(value='2021', status=True)==False
assert iyr_rule(value='2015', status=True)==True

assert eyr_rule(value='20202', status=True)==False
assert eyr_rule(value='2019', status=True)==False
assert eyr_rule(value='2020', status=True)==True
assert eyr_rule(value='2030', status=True)==True
assert eyr_rule(value='2031', status=True)==False
assert eyr_rule(value='2025', status=True)==True

assert hgt_rule(value='1T', status=True)==False
assert hgt_rule(value='aa', status=True)==False
assert hgt_rule(value='cm', status=True)==False
assert hgt_rule(value='149cm', status=True)==False
assert hgt_rule(value='194cm', status=True)==False
assert hgt_rule(value='150cm', status=True)==True
assert hgt_rule(value='193cm', status=True)==True
assert hgt_rule(value='160cm', status=True)==True
assert hgt_rule(value='58in', status=True)==False
assert hgt_rule(value='77in', status=True)==False
assert hgt_rule(value='59in', status=True)==True
assert hgt_rule(value='76in', status=True)==True
assert hgt_rule(value='70in', status=True)==True

assert hcl_rule(value='#123abc', status=True)==True
assert hcl_rule(value='#123abz', status=True)==False
assert hcl_rule(value='#123abf', status=True)==True
assert hcl_rule(value='#123abg', status=True)==False
assert hcl_rule(value='123abf', status=True)==False

assert ecl_rule(value='amb', status=True)==True
assert ecl_rule(value='blu', status=True)==True
assert ecl_rule(value='brn', status=True)==True
assert ecl_rule(value='gry', status=True)==True
assert ecl_rule(value='grn', status=True)==True
assert ecl_rule(value='hzl', status=True)==True
assert ecl_rule(value='oth', status=True)==True
assert ecl_rule(value='a', status=True)==False
assert ecl_rule(value='klgv', status=True)==False
assert ecl_rule(value='jgh', status=True)==False

assert pid_rule(value='000000001', status=True)==True
assert pid_rule(value='0123456789', status=True)==False
assert pid_rule(value='01234567', status=True)==False

In [213]:
def count_invalid_passports(passport_dict_list: [dict], cid=False) -> int:

    result = []
    passport_required_fields = ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']

    if cid:
        passport_required_fields += ['cid']
    
    for passport in tqdm(passport_dict_list):

        valid = True
        for required_field in passport_required_fields:
            
            if required_field not in passport.keys():
                valid = False
            else:
                if required_field=='byr':
                    valid = byr_rule(value=passport[required_field], 
                                     status=valid)
                elif required_field=='iyr':
                    valid = iyr_rule(value=passport[required_field], 
                                     status=valid)
                elif required_field=='eyr':
                    valid = eyr_rule(value=passport[required_field], 
                                     status=valid)
                elif required_field=='hgt':
                    valid = hgt_rule(value=passport[required_field], 
                                     status=valid)
                elif required_field=='hcl':
                    valid = hcl_rule(value=passport[required_field], 
                                     status=valid)
                elif required_field=='ecl':
                    valid = ecl_rule(value=passport[required_field], 
                                     status=valid)
                elif required_field=='pid':
                    valid = pid_rule(value=passport[required_field], 
                                     status=valid)
        result.append(valid)

    nb_of_valid = np.sum(result)
    return nb_of_valid

In [214]:
test_invalid = [{'eyr':'1972', 'cid':'100',
         'hcl':'#18171d', 'ecl':'amb', 
         'hgt':'170', 'pid':'186cm',
         'iyr':'2018', 'byr':'1926'}, 
       {'iyr':'2019', 'hcl':'#602927', 
        'eyr':'1967', 'hgt':'170cm',
        'ecl':'grn', 'pid':'012533040', 
        'byr':'1946'}, 
        {'hcl':'dab227', 'iyr':'2012',
         'ecl':'brn', 'hgt':'182cm', 
         'pid':'021572410', 'eyr':'2020', 
         'byr':'1992', 'cid':'277'}]

test_valid = [{'pid':'087499704', 'hgt':'74in', 'ecl':'grn',
               'iyr':'2012', 'eyr':'2030', 'byr':'1980', 
               'hcl':'#623a2f'}]

count_invalid_passports(passport_dict_list=test_invalid)
count_invalid_passports(passport_dict_list=test_valid)

100%|██████████| 1/1 [00:00<00:00, 9822.73it/s]


1

In [215]:
count_invalid_passports(passport_dict_list=day_4_data)

100%|██████████| 260/260 [00:00<00:00, 78167.80it/s]


179